In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/online-food-delivery-preferencesbangalore-region/onlinedeliverydata.csv")
df

In [ ]:
df.columns

In [ ]:
df["Monthly Income"].value_counts()

In [ ]:
df["Monthly Income"].values

*Changing the income strings to integers*

In [ ]:
for i in range(388):
    if df["Monthly Income"].values[i] == "No Income":
        df["Monthly Income"].values[i] = 0
    elif df["Monthly Income"].values[i] == "Below Rs.10000":
        df["Monthly Income"].values[i] = 5000
    elif df["Monthly Income"].values[i] == "10001 to 25000":
        df["Monthly Income"].values[i] = 15000
    elif df["Monthly Income"].values[i] == "25001 to 50000":
        df["Monthly Income"].values[i] = 30000
    elif df["Monthly Income"].values[i] == "More than 50000":
        df["Monthly Income"].values[i] = 60000        
df.head(11)
    

In [ ]:
df["Gender"].value_counts()

In [ ]:
df["Marital Status"].value_counts()

In [ ]:
df["Occupation"].value_counts()

# Encoding Labels

In [ ]:
from sklearn import preprocessing
le_gender = preprocessing.LabelEncoder()
le_gender.fit(['Female','Male'])
df.iloc[:,1] = le_gender.transform(df.iloc[:,1]) 


le_marital = preprocessing.LabelEncoder()
le_marital.fit(['Single', 'Married','Prefer not to say'])
df.iloc[:,2] = le_marital.transform(df.iloc[:,2])


le_occupation = preprocessing.LabelEncoder()
le_occupation.fit([ 'Student', 'Employee','Self Employeed','House wife'])
df.iloc[:,3] = le_occupation.transform(df.iloc[:,3]) 

df[0:5]

*code saving*

In [ ]:
df_encoded = df
df_encoded.head()

In [ ]:
df_encoded["Good Taste "].value_counts()

*Label encoding of the column that will be predicted(Good Taste)*

In [ ]:
le_goodtaste = preprocessing.LabelEncoder()
le_goodtaste.fit(['Very Important','Important','Moderately Important','Slightly Important','Unimportant'])
df_encoded.iloc[:,51] = le_goodtaste.transform(df_encoded.iloc[:,51]) 
df_encoded[0:5]

# Preprocessing

In [ ]:
X = df_encoded[['Age','Gender','Marital Status','Occupation','Monthly Income','Family size']].values
X[0:5]

In [ ]:
y = df_encoded['Good Taste '].values
y[0:5]

In [ ]:
X = preprocessing.StandardScaler().fit(X).transform(X.astype(float))
X[0:5]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

# K-NN classification on 'Good Taste' review of the customers

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import matplotlib.ticker as ticker
from sklearn import metrics


In [ ]:
Ks = 50
mean_acc = np.zeros((Ks-1))
std_acc = np.zeros((Ks-1))

for n in range(1,Ks):
    
    #Train Model and Predict  
    neigh = KNeighborsClassifier(n_neighbors = n).fit(X_train,y_train)
    yhat=neigh.predict(X_test)
    mean_acc[n-1] = metrics.accuracy_score(y_test, yhat)

    
    std_acc[n-1]=np.std(yhat==y_test)/np.sqrt(yhat.shape[0])

mean_acc

# Best K value

In [ ]:
plt.plot(range(1,Ks),mean_acc,'g')
plt.fill_between(range(1,Ks),mean_acc - 1 * std_acc,mean_acc + 1 * std_acc, alpha=0.10)
plt.fill_between(range(1,Ks),mean_acc - 3 * std_acc,mean_acc + 3 * std_acc, alpha=0.10,color="green")
plt.legend(('Accuracy ', '+/- 1xstd','+/- 3xstd'))
plt.ylabel('Accuracy ')
plt.xlabel('Number of Neighbors (K)')
plt.tight_layout()
plt.show()

In [ ]:
print( "The best accuracy was with", mean_acc.max(), "with k=", mean_acc.argmax()+1) 

*Here we see the best value of K is 1 for this K-NN model*...
**Thanx for reading**...